In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook

import pandas as pd
import gluonnlp as nlp
import numpy as np


from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel
from transformers import AdamW

from kobert.pytorch_kobert import get_pytorch_kobert_model

from kobert_hf.kobert_tokenizer import KoBERTTokenizer

from PIL import Image
import glob
import cv2

In [15]:
bertmodel, vocab = get_pytorch_kobert_model('C:\\Users\\cpprh/.cache\\huggingface\\transformers', '.cache')

using cached model. C:\Users\cpprh\Documents\GitHub\EatShare-AI\.cache\kobert_v1.zip
using cached model. C:\Users\cpprh\Documents\GitHub\EatShare-AI\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


NameError: name 'tokenizer' is not defined

In [19]:
from kobert_hf.kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tok=tokenizer.tokenize

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [23]:
device = torch.device("cuda:0")

max_len = 64
batch_size = 4
warmup_ratio = 0.1
num_epochs = 5  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

## 학습 모델 로드
PATH = './model/'
model = torch.load(PATH + 'KoBERT_test.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + 'model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

<All keys matched successfully>

In [26]:
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("탄2, 단2, 지2")
            elif np.argmax(logits) == 1:
                test_eval.append("탄1, 단1, 지1")
            elif np.argmax(logits) == 2:
                test_eval.append("탄0, 단0, 지0")
            elif np.argmax(logits) == 3:
                test_eval.append("탄0, 단0, 지1")
            elif np.argmax(logits) == 4:
                test_eval.append("탄0, 단1, 지0")
            elif np.argmax(logits) == 5:
                test_eval.append("탄1, 단0, 지0")
            elif np.argmax(logits) == 6:
                test_eval.append("탄0, 단1, 지1")
            elif np.argmax(logits) == 7:
                test_eval.append("탄1, 단0, 지1")
            elif np.argmax(logits) == 8:
                test_eval.append("탄1, 단1, 지0")
            elif np.argmax(logits) == 9:
                test_eval.append("탄0, 단0, 지2")
            elif np.argmax(logits) == 10:
                test_eval.append("탄0, 단2, 지0")
            elif np.argmax(logits) == 11:
                test_eval.append("탄2, 단0, 지0")
            elif np.argmax(logits) == 12:
                test_eval.append("탄0, 단1, 지2")
            elif np.argmax(logits) == 13:
                test_eval.append("탄1, 단0, 지2")
            elif np.argmax(logits) == 14:
                test_eval.append("탄0, 단2, 지1")
            elif np.argmax(logits) == 15:
                test_eval.append("탄2, 단0, 지1")
            elif np.argmax(logits) == 16:
                test_eval.append("탄2, 단1, 지0")
            elif np.argmax(logits) == 17:
                test_eval.append("탄1, 단2, 지0")
            elif np.argmax(logits) == 18:
                test_eval.append("탄1, 단1, 지2")
            elif np.argmax(logits) == 19:
                test_eval.append("탄1, 단2, 지1")
            elif np.argmax(logits) == 20:
                test_eval.append("탄2, 단1, 지1")
            elif np.argmax(logits) == 21:
                test_eval.append("탄1, 단2, 지2")
            elif np.argmax(logits) == 22:
                test_eval.append("탄2, 단1, 지2")
            elif np.argmax(logits) == 23:
                test_eval.append("탄2, 단2, 지1")
            elif np.argmax(logits) == 24:
                test_eval.append("탄0, 단2, 지2")
            elif np.argmax(logits) == 25:
                test_eval.append("탄2, 단0, 지2")
            elif np.argmax(logits) == 26:
                test_eval.append("탄2, 단2, 지0")
            else:
                test_eval.append("응애")
        print(">> " + test_eval[0])

In [27]:
while True :
    sentence = input("내용 : ")
    if sentence == "" :
        break
    predict(sentence)
    print()

내용 : 라면
>> 탄2, 단0, 지2

내용 : 우동
>> 탄2, 단0, 지0

내용 : 짬뽕
>> 탄2, 단1, 지2

내용 : 계란
>> 탄0, 단1, 지1

내용 : 비빔밥
>> 탄2, 단1, 지2

내용 : 
